https://adventofcode.com/2022/

In [1]:
import pandas as pd

# --- Day 6: Tuning Trouble ---


In [1]:
# Part 1
file = "input6.txt"
with open(file) as f:
    input_str = f.read()
input_str[:100]

'stftmtvvtvqqczqqnjnwwlqqdzdnnsvnsswbbwsstvvssfjsjbjfjmjpjzpplpppjzjqqdzzhqqqqtcccbzzzwzrrrdqdldpdspp'

In [93]:
input_str = input_str.strip("\n")

In [94]:
def seek_start_packet(input_str):
    candidate=[]
    for i, char in enumerate(input_str):
        #print(i)
        #print(f"candidate:{candidate}, char:{char}")
        candidate.append(char)
        #print(f"new candidate:{candidate}")
        input_str=input_str[1:]
        #print(f"new str:{input_str[:5]}…")
        while len(set(candidate))<len(candidate):
            candidate = candidate[1:]
        #print(f"new candidate:{candidate}")
        if len(candidate) == 4:
            return i+1
        #print("-----")

In [95]:
seek_start_packet(input_str)

1896

## Part 2

In [96]:
def seek_start_thing(input_str, len_of_searched_item=14):
    candidate=[]
    for i, char in enumerate(input_str):
        #print(i)
        #print(f"candidate:{candidate}, char:{char}")
        candidate.append(char)
        #print(f"new candidate:{candidate}")
        input_str=input_str[1:]
        #print(f"new str:{input_str[:5]}…")
        while len(set(candidate))<len(candidate):
            candidate = candidate[1:]
        #print(f"new candidate:{candidate}")
        if len(candidate) == len_of_searched_item:
            return i+1
        #print("-----")
        
input_str = input_str.strip("\n")
seek_start_thing(input_str)

3452

# --- Day 7: No Space Left On Device ---

In [2]:
# Part 1

In [3]:
file = "input7.txt"
with open(file) as f:
    input_str = f.read()

In [4]:
import re

In [16]:
# Approche liste et compteurs et récurrence

input_list = input_str.split("\n")

# if cd name:
#   next should be ls
#   then items: if file, add size to name.size
#   if dir: proceed recursively
#   if cd .. exit loop and add to dict: name: size()

def is_cd_up(item, stack, dict_folders):
    pattern = re.compile("\$ cd (\.\.)")
    re.search(pattern, item)[1]
    stack.pop()
    #print(f"item: {item}, stack: {stack}, dict_folders:{dict_folders}")
    return stack, dict_folders

    
def is_folder(item, stack, dict_folders):
    pattern = re.compile("\$ cd (.+)")
    
    dir_name = re.search(pattern, item)[1]
    #print(f"stack:{stack}")
    full_name = ""
    for n in stack[1:]:
        full_name += n; full_name+="/"
        #print(full_name)
    full_name +=dir_name
    stack.append(full_name)
    #print(stack)
    dict_folders[full_name] = 0
    #print(f"item: {item}, stack: {stack}, dict_folders:{dict_folders}")
    return stack, dict_folders

def is_file(item, stack, dict_folders):
    pattern = re.compile("(\d+?) .*")
    
    file_size = re.search(pattern, item)[1]
    #print(file_size)
    #print(stack, stack[-1], dict_folders[stack[-1]], type(dict_folders[stack[-1]]))
    #print(dict_folders)
    for i in range(len(stack)):
        dict_folders[stack[i]] += int(file_size)
    #print(f"item: {item}, stack: {stack}, dict_folders:{dict_folders}")
    return stack, dict_folders


In [17]:
stack = []
dict_folders = {}
for i in input_list:
    #print(f"####")
    
    # cd ..
    try: 
        stack, dict_folders = is_cd_up(i, stack, dict_folders)
        #print(f"this is a cd ..\n")
        continue
    except: pass #print("not a cd ..")
    
    # cd dir_name
    try: 
        stack, dict_folders = is_folder(i, stack, dict_folders)
        #print(f"this is a dir change\n")
        continue
    except: pass #print("not a dir change")
        
    # file with size
    try: 
        stack, dict_folders = is_file(i, stack, dict_folders)
        #print(f"this is a file\n")
        continue
    except: pass# print("not a file")
        
    #print("\n")

In [18]:
max_size = 100000
target_folders = [dict_folders[name] for name in dict_folders.keys() if dict_folders[name] <= max_size]

sum(target_folders) # 1543140

1543140

In [19]:
#issue
#len(input_str.split("cd ..")) > len(dict_folders)
# some subfolders may have same name

--- Part Two ---


In [20]:
total_disk_space = 70000000
needed_unused_space = 30000000

In [21]:
total_occupied_space = dict_folders['/']

In [22]:
total_occupied_space

41111105

In [23]:
free_space = total_disk_space - total_occupied_space

In [24]:
required_min_size = needed_unused_space - free_space

In [25]:
required_min_size

1111105

In [26]:
candidates = [dict_folders[k] for k in dict_folders.keys() if dict_folders[k]>=required_min_size]

In [27]:
candidates.sort()

In [28]:
candidates[0] #1117448

1117448

# --- Day 8: Treetop Tree House ---

## Part 1

In [29]:
# Consider your map; how many trees are visible from outside the grid?

# (consider only interior?)
# for each: consider each of 4 directions
# is self.height <= max height in that direction? 
# visibility = and (visibility) in each direction

file = "input8.txt"
with open(file) as f:
    input_str = f.read()
input_str[:100]

'012210012213303320303222443113334342210313303454311331225110130030424212142031302213330002110012202\n'

In [30]:
import numpy as np

input_array = np.array([[int(x) for x in list(line)] for line in input_str.split('\n')[:-1]])

input_array.shape

(99, 99)

In [31]:
def is_visible(n_line, n_col, forest):
    
    height=forest[n_line,n_col]
    line=forest[n_line,:]
    col =forest[:,n_col]
    #print(height)
    #print(line)
    #print(col)

    try: 
        my_max=max(line[:n_col])
        is_visible_from_left= my_max < height
    except: is_visible_from_left = True 
    #print(f"is_visible_from_left {is_visible_from_left}")

    try: 
        my_max=max(line[n_col+1:])
        is_visible_from_right= my_max < height
    except: is_visible_from_right=True
    #print(f"is_visible_from_right {is_visible_from_right}")
    
    try: 
        my_max=max(col[:n_line])
        is_visible_from_top= my_max < height
    except: is_visible_from_top=True
    #print(f"is_visible_from_top {is_visible_from_top}")
    
    try: 
        my_max=max(col[n_line+1:])
        is_visible_from_bottom= my_max < height
    except: is_visible_from_bottom=True
    #print(f"is_visible_from_bottom {is_visible_from_bottom}")
    
    return int(is_visible_from_left or is_visible_from_right or is_visible_from_top or is_visible_from_bottom)

In [32]:
visible=np.array([[is_visible(i,j, input_array) for j in range(input_array.shape[1])] for i in range(input_array.shape[0])])
;

''

In [33]:
sum(sum(visible)) # 1825

1825

In [34]:
def scenic_score_partial(height_tree, list_from_nth):
        #print(f"height of tree:{height_tree}, list_from_nth:{list_from_nth}")        
        if len(list_from_nth)==0: return 0
        for n, height_neighbor in enumerate(list_from_nth):
            #print(f"n:{n}, height_neighbor:{height_neighbor}, height_tree:{height_tree}")
            if height_neighbor >= height_tree:
                #print(f"all ok, dist={n+1}")
                return n+1
        else: return len(list_from_nth)
            
scenic_score_partial(3, [1,3,3,8])

2

In [35]:
def scenic_score(n_line, n_col, forest):
    
    height=forest[n_line,n_col]
    line=forest[n_line,:]
    col =forest[:,n_col]
    #print(height)
    #print(line)
    #print(col)

    if n_col==0: list_left = [] 
    else: list_left =line[n_col-1::-1]
    #print(f"🔵line:{line}\nlist_left:{list_left}")
    scenic_score_left = scenic_score_partial(height, list_left) 
    #print(f"scenic_score_left {scenic_score_left}")

    list_right = line[n_col+1:]
    #print(list_right)
    scenic_score_right = scenic_score_partial(height, list_right) 
    #print(f"scenic_score_right {scenic_score_right}")
    
    if n_line==0: list_top = []
    else: list_top =col[n_line-1::-1]
    #print(f"🔵col:{col}\nlist_top:{list_top}")
    scenic_score_top = scenic_score_partial(height, list_top) 
    #print(f"scenic_score_top {scenic_score_top}")

    list_bottom = col[n_line+1:]
    #print(list_bottom)
    scenic_score_bottom = scenic_score_partial(height, list_bottom) 
    #print(f"scenic_score_bottom {scenic_score_bottom}")
    
    #return max(scenic_score_left, scenic_score_right, scenic_score_top, scenic_score_bottom)

    return (scenic_score_left * scenic_score_right * scenic_score_top * scenic_score_bottom)
    #print(scenic_score_left, scenic_score_right, scenic_score_top, scenic_score_bottom)

In [36]:
scenic_score(2,4, input_array)

1

In [37]:
#Consider each tree on your map. What is the highest scenic score possible for any tree?

scenic=np.array([[scenic_score(i,j, input_array) for j in range(input_array.shape[1])] for i in range(input_array.shape[0])])

In [38]:
scenic.max(axis=0).max() #235200

235200

In [243]:
#compare 44*44*44*44 #3748096

# --- Day 9: Rope Bridge ---



In [44]:
file="input9.txt"
with open(file) as f:
    input_str= f.read()
input_str[:100]

'L 1\nR 1\nD 1\nR 2\nD 2\nL 1\nD 1\nL 2\nR 1\nL 1\nU 1\nD 2\nR 1\nD 2\nR 1\nL 2\nD 1\nU 1\nL 2\nU 1\nD 1\nU 1\nD 2\nL 2\nD 2\n'

In [45]:
input_moves=input_str.split("\n")[:-1] # last is ''
import re
pattern = re.compile("([LRDU]) (\d)")
moves = [(re.search(pattern, string)[1], re.search(pattern, string)[2]) for string in input_moves]

In [46]:
moves[:4]

[('L', '1'), ('R', '1'), ('D', '1'), ('R', '2')]

In [47]:
import math
import numpy as np

def euclidean_distance(current_pos_H, current_pos_T):
  return math.sqrt((current_pos_H[0] - current_pos_T[0])**2 + (current_pos_H[1] - current_pos_T[1])**2)

def follow_coord(one_coord_H, one_coord_T):
    """ one_coord_H and one_coord_T are integer coordinates, either both x or both y"""
    if one_coord_H == one_coord_T: return one_coord_T
    else: return one_coord_T+np.sign(one_coord_H - one_coord_T)

def follow(current_pos_H, current_pos_T, coord_T=[0,0]):
    """ both arguments are (x,y) tuples"""
    if (abs(current_pos_H[0]-current_pos_T[0])<=1 and abs(current_pos_H[1]-current_pos_T[1])<=1): 
        return current_pos_T
    for i in range(2):
        coord_T[i]=follow_coord(current_pos_H[i],current_pos_T[i])
    #print(f"follow gave: {coord_T}")
    return tuple(coord_T)

def move_in_direction(direction, current_pos_H):
    if direction == 'L': return [current_pos_H[0]-1, current_pos_H[1]]
    if direction == 'R': return [current_pos_H[0]+1, current_pos_H[1]]
    if direction == 'D': return [current_pos_H[0], current_pos_H[1]-1]
    if direction == 'U': return [current_pos_H[0], current_pos_H[1]+1]

In [50]:
def perform_move(move_H, pos_H, pos_T, all_T_pos):
    """move is of form: ('L', '1')
    pos_H and pos_T are list of [x,y] lists, latest being current position"""
    # while move_H.nb >0: first T follows, then H moves
    
    direction = move_H[0]
    nb_moves = int(move_H[1])
    current_pos_H = pos_H[-1]
    current_pos_T = pos_T[-1]
    
    #print(f"current_pos_T:{current_pos_T}")
        
    for i in range(nb_moves):
        #print(f"move {move_H}, step #{i+1}")
                
        # move H
        current_pos_H = move_in_direction(direction, current_pos_H)
        #print(f"new current_pos_H:{current_pos_H}")

        # follow T
        current_pos_T = follow(current_pos_H, current_pos_T)
        # update all_T_pos
        all_T_pos.append(current_pos_T)
        #print(f"all_T_pos: {all_T_pos}\n")
        

    # update positions
    pos_H.append(tuple(current_pos_H))
    pos_T.append(tuple(current_pos_T))
    
    #print(f"new pos_H: {pos_H},\npos_T:{pos_T}\n\n\n")
    
    return pos_H, pos_T, all_T_pos   

In [51]:
pos_H = [(0, 0)]
pos_T = pos_H.copy()
all_T_pos = pos_H.copy()
i=0


test_moves_str = """R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
"""

test_input_moves=test_moves_str.split("\n")[:-1] # last is ''
import re
pattern = re.compile("([LRDU]) (\d)")
test_moves = [(re.search(pattern, string)[1], re.search(pattern, string)[2]) for string in test_input_moves]

for i in range (len(test_moves)):
    move_H=test_moves[i]
    #print(f"### before move \npos_H: {pos_H},\npos_T:{pos_T}\n\n")
    pos_H, pos_T, all_T_pos = perform_move(move_H, pos_H, pos_T, all_T_pos)
    #print(f"### now pos_H: {pos_H},\npos_T:{pos_T}")
    i += 1
    
len(set(all_T_pos))

13

In [52]:
set(all_T_pos)

{(0, 0),
 (1, 0),
 (1, 2),
 (2, 0),
 (2, 2),
 (2, 4),
 (3, 0),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 1),
 (4, 2),
 (4, 3)}

In [53]:
pos_H = [(0, 0)]
pos_T = pos_H.copy()
all_T_pos = pos_H.copy()
i=0

for i in range (len(moves)):
    move_H=moves[i]
    #print(f"### before move \npos_H: {pos_H},\npos_T:{pos_T}\n\n")
    pos_H, pos_T, all_T_pos = perform_move(move_H, pos_H, pos_T, all_T_pos)
    #print(f"### now pos_H: {pos_H},\npos_T:{pos_T}")
    i += 1
    
len(all_T_pos) #6839

6839

In [54]:
len(set(all_T_pos)) #3196

3196

In [55]:
len(set([str(i[0])+";"+str(i[1]) for i in all_T_pos]))
#set seems as good on tuples as on string

3196

In [56]:
len(set(all_T_pos))

3196

In [61]:
#used most elegant solution from reddit https://www.reddit.com/r/adventofcode/comments/zgnice/2022_day_9_solutions/ errop_
from math import sqrt

UNIT_VECTORS = {"U": 1j, "D": -1j, "L": -1, "R": 1}

def move_tail(moves: list, length: int): 
    rope = [0j] * length 
    tail_positions = set() 
    for direction, steps in moves: 
        for _ in range(int(steps)): 
            rope[0] += UNIT_VECTORS[direction] 
            for n in range(1, len(rope)): 
                diff = rope[n - 1] - rope[n] 
                if abs(diff) > sqrt(2): 
                    if diff.real != 0: 
                        rope[n] += diff.real / abs(diff.real) 
                    if diff.imag != 0: 
                        rope[n] += complex(0, diff.imag) / abs(diff.imag) 
            tail_positions.add(rope[-1]) 
    return tail_positions

with open("input9.txt") as f: 
    moves = [l.split() for l in f.read().splitlines()]

# PART 1
print(len(move_tail(moves, 2))) #3196

# PART 2
print(len(move_tail(moves, 10))) #3196

6269
2557


# --- Day 10: Cathode-Ray Tube ---

In [39]:
# part 1

file="input10.txt"
with open(file) as f:
    input_str=f.read()
input_str[:100]

'addx 1\naddx 5\nnoop\naddx -1\nnoop\naddx 3\naddx 29\naddx -1\naddx -21\naddx 5\nnoop\naddx -20\naddx 21\naddx 2\n'

In [41]:
ops=input_str.split("\n")[:-1]

def parse_op(op):
    #print(op, len(op.split()))
    if len(op.split())==1:
        op=[1,0]
    else:
        op=[2, int(op.split()[1])]
    return op
        

ops=[parse_op(op) for op in ops]
ops[:15]


[[2, 1],
 [2, 5],
 [1, 0],
 [2, -1],
 [1, 0],
 [2, 3],
 [2, 29],
 [2, -1],
 [2, -21],
 [2, 5],
 [1, 0],
 [2, -20],
 [2, 21],
 [2, 2],
 [2, 8]]

In [42]:
stops=[20, 60, 100, 140,180, 220]
X=1
cycle=1
draws={}
    
def check_stops(cycle, stops, X):
    if cycle in stops:
        draws[cycle]=X        
        

for op in ops:
    #print(f"cycle:{cycle}, op:{op}, X:{X}")
    for i in range(op[0]):
        #print(i)
        check_stops(cycle, stops, X)
        cycle+=1
    X+=op[1]

In [8]:
draws

{20: 21, 60: 17, 100: 21, 140: 17, 180: 21, 220: 37}

In [10]:
sum([k * v for (k, v) in draws.items()]) #17840

17840

In [43]:
# part 2

X=1
cycle=1
draws={}
CRT_position=0
lights=""

for op in ops:
    #print(f"Start cycle   {cycle}, begin executing op:{op}, X:{X}")
    for i in range(op[0]):
        #print(f"During cycle {cycle}: CRT draws pixel in position {CRT_position}")
        if X%40 > CRT_position+1 or X%40 < CRT_position-1:
            lights+='.'
        else:
            lights+="#"
        cycle+=1
        
        CRT_position+=1
        if CRT_position==40:
            lights+="\n"
        CRT_position=CRT_position%40

        #print(lights+"\n")
    X+=op[1]

In [57]:
i=0
len(lights.splitlines()[i])

40

In [59]:
for line in lights.splitlines():
    print(line)


####..##..#.....##..#..#.#....###...##.#
.....#..#.#....#..#.#..#.#....#..#.#..#.
###..#..#.#....#....#..#.#....#..#.#..##
.....####.#....#.##.#..#.#....###..#.###
.....#..#.#....#..#.#..#.#....#....#..#.
####.#..#.####..###..##..####.#.....###.
